In [1]:
from numpy.random import seed
seed(1)
import tensorflow 
tensorflow.random.set_seed(1) 
import pandas as pd


2022-07-05 16:37:03.762743: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-05 16:37:03.762759: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
pip install scikeras

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
[[60]]

[[60]]

In [4]:
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to /home/alice/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
from sklearn.model_selection import train_test_split

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
from imblearn.over_sampling import SMOTE
from collections import Counter

In [8]:
from tensorflow import keras

In [9]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv1D, MaxPooling1D, Convolution1D
from keras.optimizers import Adadelta

In [10]:
from sklearn import metrics

In [11]:
def ageGroups(text):
    if text == '10 à 18 anos' or text=='19 à 25 anos':
        return 0
    elif text=='26 à 30 anos' or text=='30 à 40 anos':
        return 1
    else:
        return 2

In [12]:
def remove_stopwords(text):
    all_words = text.split(" ")
    clean_text = [i for i in all_words if i not in stopwords and i!=""]
    return " ".join(clean_text)

In [13]:
def remove_html_tags(text):
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [14]:
def lower_texts(text):
    return text.lower()

In [15]:
def clean_text(text):
    clean_text = remove_stopwords(text)
    clean_text = remove_html_tags(clean_text)
    clean_text = lower_texts(clean_text)
    return clean_text

In [16]:
def genderClassifier(text):
    if text == 'Masculino':
        return 0
    else:
        return 1

In [17]:
from scikeras.wrappers import KerasClassifier

In [18]:
from sklearn.model_selection import GridSearchCV

In [19]:
def create_model(size, n_class):
    model = Sequential()
    model.add(Conv1D(filters = 100, 
                        kernel_size = 3,
                        strides = 1, 
                        activation = 'relu', 
                        input_shape = (size, 1) ))
    model.add(MaxPooling1D(pool_size = 4, strides = 1))
    model.add(Activation('relu')) # add layer
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(units = 512, activation = 'relu')) #add relu
    model.add(Dense(units = n_class, activation = 'softmax'))

    model.compile(loss='categorical_crossentropy', optimizer = Adadelta(
        learning_rate=1, name="Adadelta"
    ), metrics = ['accuracy'])
    return model

In [22]:
df_train = pd.read_csv("/home/alice/Pesquisa/tcc-v2/particoes/blogsetbr/k_1/train.csv")
df_test = pd.read_csv("/home/alice/Pesquisa/tcc-v2/particoes/blogsetbr/k_1/test.csv")
X_train = df_train["Texts"].apply(clean_text).to_numpy()
X_test = df_test["Texts"].apply(clean_text).to_numpy()
y_train = df_train["GenderClass"].to_numpy()
y_test = df_test["GenderClass"].to_numpy()
num_words = []
for text in (X_train.tolist()+X_test.tolist()):
    num_words.append(len(text.split()))
mean = sum(num_words)//len(num_words)
train_texts = X_train.tolist()
test_texts = X_test.tolist()
tfidfvec = TfidfVectorizer(max_features = mean, max_df=0.9)
tfidfvec.fit(train_texts)
tfidf_train = tfidfvec.transform(train_texts).toarray()
tfidf_test = tfidfvec.transform(test_texts).toarray()
X_train = tfidf_train.reshape(tfidf_train.shape[0],tfidf_train.shape[1],1)
X_test = tfidf_test.reshape(tfidf_test.shape[0],tfidf_test.shape[1],1)
y_train = keras.utils.to_categorical(y_train,num_classes=2)
y_test = keras.utils.to_categorical(y_test,num_classes=2)

size = X_test.shape[1]

model = KerasClassifier(model=create_model, size=X_test.shape[1], n_class=2, verbose=0)

batch_size = [32, 64]
epochs = [1, 10]
param_grid = dict(batch_size=batch_size, epochs=epochs)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, scoring='f1_macro')
grid_result = grid.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

2022-07-05 17:17:28.237079: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-05 17:17:28.237097: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-07-05 17:17:28.783964: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-05 17:17:28.784170: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-07-05 17:17:28.785270: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or dire

Best: 0.707113 using {'batch_size': 64, 'epochs': 10}
